In [ ]:
import torch
from heat_solver import HeatSolver
from inverse_solver import InverseSolver
from utils import create_conductivity_field, sine_source
from utils import SimpleSigma, SigmoidSigma

from run import get_boundary_conditions